In [ ]:
import pandas as pd
#from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPool2D, AveragePooling2D, Input, BatchNormalization, MaxPooling2D, Activation, Flatten, Dense, Dropout
from keras.models import Model
from keras.utils import to_categorical
from sklearn.metrics import classification_report
#from imblearn.over_sampling import RandomOverSampler
from keras.preprocessing import image
import scipy
import os
import cv2

In [ ]:
data = pd.read_csv('/content/drive/My Drive/icml_face_data.csv')

In [ ]:
data.head()

In [ ]:
data

In [ ]:
pixel_data = data[' pixels']
label_data = data['emotion']

In [ ]:
def preprocess_pixels(pixel_data):
  images = []
  for i in range(len(pixel_data)):
    img = np.fromstring(pixel_data[i], dtype='int', sep=' ')
    img = img.reshape(48,48,1)
    images.append(img)

  X = np.array(images)

 

  return X

In [ ]:
oversampler = RandomOverSampler(sampling_strategy='auto')

X_over, Y_over = oversampler.fit_resample(pixel_data.values.reshape(-1,1), label_data)

In [ ]:
X_over_series = pd.Series(X_over.flatten())
X_over_series

In [ ]:
X = preprocess_pixels(X_over_series)
Y = Y_over

In [ ]:
Y = Y_over.reshape(Y.shape[0],1)
Y.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 45)

In [ ]:
def emotion_recognition(input_shape):

  X_input = Input(input_shape)

  X = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='valid')(X_input)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)


  X = Conv2D(64, (3,3), strides=(1,1), padding = 'same')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)

  X = MaxPooling2D((2,2))(X)

  X = Conv2D(64, (3,3), strides=(1,1), padding = 'valid')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)

  X = Conv2D(128, (3,3), strides=(1,1), padding = 'same')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)


  X = MaxPooling2D((2,2))(X)

  X = Conv2D(128, (3,3), strides=(1,1), padding = 'valid')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)

 

  X = MaxPooling2D((2,2))(X)
  X = Flatten()(X)
  X = Dense(200, activation='relu')(X)
  X = Dropout(0.6)(X)
  X = Dense(7, activation = 'softmax')(X)

  model = Model(inputs=X_input, outputs=X)

  return model

In [ ]:
model = emotion_recognition((48,48,1))

In [ ]:
model.summary()

In [ ]:
adam = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
y_train = to_categorical(Y_train, num_classes=7)

In [ ]:
y_train.shape

In [ ]:
y_test = to_categorical(Y_test, num_classes=7)

In [ ]:
model.fit(X_train, y_train, epochs = 30, validation_data=(X_test, y_test))

In [ ]:
model.save_weights('/content/drive/My Drive/emotion_weights_3.hdf5')

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
preds = model.predict(X_train)

In [ ]:
def get_class(preds):
  pred_class = np.zeros((preds.shape[0],1))

  for i in range(len(preds)):
   pred_class[i] = np.argmax(preds[i])

  return pred_class

In [ ]:
pred_class_train = get_class(preds)

In [ ]:
train_report = classification_report(Y_train, pred_class_train)

In [ ]:
print(train_report)

In [ ]:
test_preds = model.predict(X_test)

In [ ]:
pred_test_class = get_class(test_preds)

In [ ]:
report_test = classification_report(Y_test, pred_test_class)
print(report_test)

In [ ]:
label_dict = {0 : 'Angry', 1 : 'Disgust', 2 : 'Fear', 3 : 'Happiness', 4 : 'Sad', 5 : 'Surprise', 6 : 'Neutral'}
label_dict

In [ ]:
img_path = 'test1_happiness.jpeg'
img = image.load_img(img_path, grayscale=True, target_size=(48,48))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x.shape

In [ ]:
prediction = np.argmax(model.predict(x))
print('The predicted emotion is : ' + label_dict[prediction])
my_image = image.load_img(img_path)
plt.imshow(my_image)

In [ ]:
model.load_weights('C:\\Users\\avneet\\Desktop\\Machine Learning\\Facial Expression\\emotion_weights_3.hdf5')

In [ ]:
face_haar_cascade = cv2.CascadeClassifier('C:\\Users\\avneet\\Desktop\\Machine Learning\\Facial Expression\\archive\\haarcascade_frontalface_default.xml'

In [ ]:
cap = cv2.VideoCapture(0)

while True:
 
 _,cap_image = cap.read()

    
    

 cap_img_gray = cv2.cvtColor(cap_image, cv2.COLOR_BGR2GRAY)

 faces = face_haar_cascade.detectMultiScale(cap_img_gray, 1.3, 5)

 for (x,y,w,h) in faces:

   cv2.rectangle(cap_image, (x,y), (x+w,y+h),(255,0,0),2)
   roi_gray = cap_img_gray[y:y+h, x:x+w]
   roi_gray = cv2.resize(roi_gray, (48,48))
   img_pixels = image.img_to_array(roi_gray)
   img_pixels = np.expand_dims(img_pixels, axis=0)

   predictions = model.predict(img_pixels)
   emotion_label = np.argmax(predictions)

   emotion_prediction = label_dict[emotion_label]

   cv2.putText(cap_image, emotion_prediction, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1 )

   resize_image = cv2.resize(cap_image, (1000,700))
   cv2.imshow('Emotion',resize_image)

   if cv2.waitKey(10) == ord('b'):
    break
cap.release()
cv2.destroyAllWindows 